# Python &rarr; OKL Sandbox

AST Grammar info can be found in the [ast docs](https://docs.python.org/3/library/ast.html#abstract-grammar)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import List
import ast
import inspect

from occa import okl
from occa.okl import Const, Exclusive, Shared
from occa.okl import Oklifier, py2okl

In [9]:
def add(a: Const[float], 
        b: Const[float]) -> float:
    return a + b

@okl.kernel
def add_vectors(a: Const[List[float]],
                b: Const[List[float]],
                ab: List[float]) -> None:
    for i in okl.range(len(a)).tile(16):
        # Type tests
        foo: bool = True
        bar: np.float64 = 3.2
        s_foo: Shared[List[float, 20, 30]]
        e_foo: Exclusive[float]
        bar: Const[List[float, 2]] = [1,2]
            
        s: str = 'string'
            
        # Flow tests
        for j in range(10):
            pass
        
        if i > 10:
            continue
        elif i < 5:
            break
        else:
            i: int = 3
            return 3
        
        while True:
            pass
            
        # Closure tests
        ab[i] = add(a[i], b[i])

In [10]:
print(add_vectors.__okl_source__)

double add(const double a,
           const double b);

double add(const double a,
           const double b) {
  return a + b;
}

@kernel void add_vectors(const double *a,
                         const double *b,
                         double *ab) {
  for (int i = 0; i < 10; ++i) {
    bool foo = true;
    double bar = 3.2;
    @shared double s_foo[20][30];
    @exclusive double e_foo;
    const double bar[2] = {1, 2};
    char * s = "string";
    for (int j = 0; j < 10; ++j) {}
    if (i > 10) {
      continue;
    }
    else if (i < 5) {
      break;
    }
    else  {
      int i = 3;
      return 3;
    }
    while (true) {}
    ab[i] = add(a[i], b[i]);
  }
}


In [5]:
foo = okl.Oklifier.get_last_error_node()

In [6]:
foo

In [37]:
py2okl(foo.value.value)

'np'

In [36]:
foo.value.attr

'a'

In [61]:
py2okl.py2okl(foo.annotation)

'float'

In [167]:
py2okl.py2okl(iter)

'okl.range(len(a)).tile(foo)'

In [132]:
py2okl.py2okl(iter.args)

['16']

In [136]:
iter.func.attr

'tile'

In [137]:
py2okl.py2okl(iter.func.value)

'okl.range(len(a))'

In [138]:
py2okl.py2okl(iter.func.value.args)

['len(a)']

In [141]:
iter.func.value.func.attr

'range'

In [143]:
py2okl.py2okl(iter.func.value.func.value)

'okl'

In [144]:
py2okl.py2okl(iter.func.value.func)

'okl.range'

In [145]:
py2okl.py2okl(iter.func)

'okl.range(len(a)).tile'